In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.utils import save_image
import os
from PIL import Image

img_size = 128
batch_size = 1
lr = 0.0002
epochs = 200 

output_dir = "cyclegan_generated"
os.makedirs(output_dir, exist_ok=True)

class UnpairedImageDataset(Dataset):
    def __init__(self, path1, path2, transform=None):
        self.images1 = [os.path.join(path1, img) for img in os.listdir(path1)]
        self.images2 = [os.path.join(path2, img) for img in os.listdir(path2)]
        self.transform = transform

    def __len__(self):
        return min(len(self.images1), len(self.images2))

    def __getitem__(self, idx):
        img1 = Image.open(self.images1[idx % len(self.images1)]).convert('RGB')
        img2 = Image.open(self.images2[idx % len(self.images2)]).convert('RGB')

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return img1, img2

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

path1 = 'D:\Gan\dataset1'
path2 = 'D:\Gan\dataset2'
dataset = UnpairedImageDataset(path1, path2, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3),
            nn.ReLU(True),
            nn.Conv2d(64, 3, kernel_size=7, stride=1, padding=3),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(128, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
G_AB = Generator().to(device)
G_BA = Generator().to(device)
D_A = Discriminator().to(device)
D_B = Discriminator().to(device)

optim_G = optim.Adam(list(G_AB.parameters()) + list(G_BA.parameters()), lr=lr, betas=(0.5, 0.999))
optim_D_A = optim.Adam(D_A.parameters(), lr=lr, betas=(0.5, 0.999))
optim_D_B = optim.Adam(D_B.parameters(), lr=lr, betas=(0.5, 0.999))

criterion_gan = nn.MSELoss()
criterion_cycle = nn.L1Loss()

for epoch in range(epochs):
    for img_A, img_B in dataloader:
        img_A, img_B = img_A.to(device), img_B.to(device)

        fake_B = G_AB(img_A)
        fake_A = G_BA(img_B)

        recon_A = G_BA(fake_B)
        recon_B = G_AB(fake_A)

        D_B_fake = D_B(fake_B)
        D_A_fake = D_A(fake_A)
        loss_GAN_AB = criterion_gan(D_B_fake, torch.ones_like(D_B_fake))
        loss_GAN_BA = criterion_gan(D_A_fake, torch.ones_like(D_A_fake))

        loss_cycle_A = criterion_cycle(recon_A, img_A)
        loss_cycle_B = criterion_cycle(recon_B, img_B)

        loss_G = loss_GAN_AB + loss_GAN_BA + 10 * (loss_cycle_A + loss_cycle_B)

        optim_G.zero_grad()
        loss_G.backward()
        optim_G.step()

        D_A_real = D_A(img_A)
        D_A_fake = D_A(fake_A.detach())
        loss_D_A = criterion_gan(D_A_real, torch.ones_like(D_A_real)) + criterion_gan(D_A_fake, torch.zeros_like(D_A_fake))

        optim_D_A.zero_grad()
        loss_D_A.backward()
        optim_D_A.step()

        D_B_real = D_B(img_B)
        D_B_fake = D_B(fake_B.detach())
        loss_D_B = criterion_gan(D_B_real, torch.ones_like(D_B_real)) + criterion_gan(D_B_fake, torch.zeros_like(D_B_fake))

        optim_D_B.zero_grad()
        loss_D_B.backward()
        optim_D_B.step()

    if (epoch + 1) % 10 == 0:
        save_image(fake_B, f"{output_dir}/fake_B_epoch_{epoch+1}.png")
        save_image(fake_A, f"{output_dir}/fake_A_epoch_{epoch+1}.png")
        print(f"Epoch [{epoch+1}/{epochs}] Loss G: {loss_G:.4f}, Loss D_A: {loss_D_A:.4f}, Loss D_B: {loss_D_B:.4f}")

print("Training complete!")


Epoch [10/200] Loss G: 1.8094, Loss D_A: 0.2462, Loss D_B: 0.2641
Epoch [20/200] Loss G: 1.9074, Loss D_A: 0.1504, Loss D_B: 0.1682
Epoch [30/200] Loss G: 2.0068, Loss D_A: 0.1141, Loss D_B: 0.1349
Epoch [40/200] Loss G: 2.1184, Loss D_A: 0.1172, Loss D_B: 0.1173
Epoch [50/200] Loss G: 2.1640, Loss D_A: 0.1222, Loss D_B: 0.1113
Epoch [60/200] Loss G: 2.2276, Loss D_A: 0.1196, Loss D_B: 0.1004
Epoch [70/200] Loss G: 2.1699, Loss D_A: 0.0731, Loss D_B: 0.0725
Epoch [80/200] Loss G: 2.2643, Loss D_A: 0.0575, Loss D_B: 0.0703
Epoch [90/200] Loss G: 2.2235, Loss D_A: 0.0601, Loss D_B: 0.0531
Epoch [100/200] Loss G: 2.2404, Loss D_A: 0.0462, Loss D_B: 0.0448
Epoch [110/200] Loss G: 2.2274, Loss D_A: 0.0357, Loss D_B: 0.0332
Epoch [120/200] Loss G: 2.2280, Loss D_A: 0.0511, Loss D_B: 0.0355
Epoch [130/200] Loss G: 2.2546, Loss D_A: 0.0496, Loss D_B: 0.0291
Epoch [140/200] Loss G: 2.2326, Loss D_A: 0.0384, Loss D_B: 0.0403
Epoch [150/200] Loss G: 2.2170, Loss D_A: 0.0429, Loss D_B: 0.0247
Epoc